In [25]:
import sklearn
import pandas as pd
import nltk
import numpy as np
import spacy
import re
from nltk.corpus import words, stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import metrics
from tqdm.auto import tqdm
tqdm.pandas()  # Enable tqdm for pandas

In [26]:
#import data
df = pd.read_csv("rest_data_last.csv")
print(df.head())

  category                                               text
0     arts  rob delaney vir das galen hopper samson kayo g...
1     arts  andris nelsons conducts a joint concert of the...
2     arts  warner music group has brought on sherry tan t...
3     arts  adele will explore what she s been going throu...
4     arts  you are using an older browser version. please...


In [27]:
import nltk
from nltk.corpus import words, stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter

nltk.download("punkt_tab")
nltk.download("words")
nltk.download("wordnet")
nltk.download("stopwords")

# Load resources
english_vocab = set(words.words())
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english")) | {"said", "year", "one", "also", "time", "like",
                                                "new", "people", "state", "june", "say", "get",
                                                "may", "many", "would", "day", "year", "two",
                                                "last", "three", "first", "home", "city", "jun",
                                                "could", "country", "county", "child", "care"}

def lemmatize_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    tokens = [word for word in tokens if len(word) >= 3]
    tokens = [word for word in tokens if word in english_vocab]
    return " ".join(tokens)

df["clean_text"] = df["text"].apply(lemmatize_text)

from collections import Counter

# assuming df is your DataFrame
def most_common_words(df, category_col, text_col, top_n=25):
    # Group by category and apply word count
    word_counts = df.groupby(category_col)[text_col].apply(
        lambda x: ' '.join(x).lower()
    ).reset_index()

    # Get top N words for each category
    top_words = []
    for index, row in word_counts.iterrows():
        category = row[category_col]
        text = row[text_col]
        words = re.findall(r'\b\w+\b', text)
        counter = Counter(words)
        top_n_words = counter.most_common(top_n)
        top_words.append((category, top_n_words))

    return top_words

result = most_common_words(df, 'category', 'clean_text')
for category, words in result:
    print(f"Category: {category}")
    for word, count in words:
        print(f"{word}: {count}")
    print()

result = most_common_words(df, 'category', 'clean_text')
# for category, words in result:
#     print(f"Category: {category}")
#     for word, count in words:
#         print(f"{word}: {count}")
#     print()

# for category, words in result:
#     print(f"category: {category}")
#     for word, count in words:
#         print(f"{word}: {count}")
#     print()

# for category, words in result:
#     print(f"'{category}':[")
#     for word, count in words:
#         print(f"'{word}',", end='')
#     print(']')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Category: arts
music: 297
show: 283
event: 252
series: 213
park: 190
group: 168
best: 168
take: 166
back: 162
year: 161
life: 160
star: 159
community: 159
million: 157
art: 153
make: 151
way: 150
book: 149
season: 148
school: 148
work: 143
rock: 134
world: 133
road: 132
see: 130

Category: crime
police: 607
court: 406
right: 295
case: 278
law: 277
officer: 251
year: 201
member: 181
charge: 175
crime: 174
prison: 173
old: 173
death: 166
found: 166
hearing: 165
justice: 159
security: 151
drug: 151
federal: 145
man: 142
report: 141
told: 138
prosecutor: 136
group: 133
president: 133

Category: disaster
fire: 620
water: 398
area: 242
police: 222
air: 211
rescue: 182
service: 164
community: 158
official: 145
year: 143
road: 141
department: 140
near: 140
report: 138
crew: 138
flight: 137
car: 137
help: 135
according: 132
ship: 127
national: 126
around: 126
vehicle: 126
week: 126
show: 122

Category: economy
company: 381
price: 276
market: 254
per: 223
million: 214
tax: 198
store: 186
product

In [28]:
checkWords = {
    'arts': [
        'music', 'show', 'event', 'series', 'park', 'group', 'best', 'take', 'year', 'star', 'back', 'life', 'million',
        'community', 'work', 'make', 'way', 'art', 'book', 'school', 'season', 'world', 'road', 'rock', 'know'
    ],
    'crime': [
        'police', 'court', 'right', 'law', 'case', 'officer', 'year', 'member', 'charge', 'crime', 'prison', 'old',
        'found', 'death', 'hearing', 'justice', 'security', 'federal', 'prosecutor', 'drug', 'group', 'told', 'report',
        'man', 'president'
    ],
    'disaster': [
        'fire', 'water', 'area', 'police', 'air', 'rescue', 'service', 'community', 'official', 'flight', 'year',
        'road', 'near', 'department', 'report', 'crew', 'help', 'car', 'according', 'ship', 'around', 'say', 'vehicle',
        'week', 'national'
    ],
    'economy': [
        'company', 'price', 'market', 'per', 'million', 'tax', 'store', 'product', 'share', 'year', 'business',
        'industry', 'billion', 'service', 'month', 'group', 'government', 'plan', 'rate', 'investment', 'cent',
        'report', 'since', 'statement', 'increase'
    ],
    'education': [
        'school', 'student', 'university', 'education', 'high', 'college', 'teacher', 'program', 'community',
        'learning', 'class', 'year', 'work', 'support', 'help', 'degree', 'need', 'member', 'board', 'government',
        'president', 'child', 'district', 'covid', 'service'
    ],
    'environmental': [
        'park', 'energy', 'water', 'waste', 'power', 'company', 'market', 'project', 'plant', 'group', 'area', 'north',
        'management', 'solar', 'plan', 'river', 'need', 'wind', 'industry', 'renewable', 'community', 'local', 'year',
        'system', 'world'
    ],
    'health': [
        'health', 'covid', 'patient', 'vaccine', 'hospital', 'market', 'case', 'medical', 'service', 'disease',
        'pandemic', 'week', 'food', 'company', 'help', 'system', 'month', 'report', 'public', 'need', 'virus',
        'global', 'work', 'well', 'government'
    ],
    'humanInterest': [
        'award', 'plant', 'question', 'team', 'size', 'winner', 'act', 'statement', 'back', 'winning', 'win', 'flower',
        'game', 'best', 'security', 'second', 'service', 'year', 'good', 'world', 'well', 'ownership', 'high',
        'beneficial', 'top'
    ],
    'labour': [
        'work', 'employee', 'worker', 'health', 'job', 'security', 'business', 'working', 'retirement', 'need',
        'pandemic', 'company', 'make', 'covid', 'plan', 'benefit', 'help', 'month', 'service', 'week', 'union',
        'social', 'market', 'office', 'employer'
    ],
    'lifestyle': [
        'product', 'travel', 'make', 'need', 'bridge', 'summer', 'life', 'beauty', 'business', 'customer', 'brand',
        'food', 'want', 'wedding', 'restaurant', 'way', 'month', 'take', 'work', 'service', 'back', 'million', 'come',
        'year', 'covid'
    ],
    'other': [
        'game', 'market', 'covid', 'school', 'government', 'week', 'team', 'rub', 'make', 'high', 'second', 'season',
        'report', 'company', 'business', 'pandemic', 'health', 'area', 'need', 'help', 'case', 'year', 'back',
        'service', 'contact'
    ],
    'politics': [
        'government', 'president', 'minister', 'security', 'data', 'public', 'service', 'right', 'official',
        'international', 'support', 'state', 'community', 'year', 'election', 'national', 'help', 'news', 'united',
        'plan', 'group', 'law', 'covid', 'house', 'pandemic'
    ],
    'religion': [
        'church', 'life', 'school', 'family', 'community', 'year', 'world', 'way', 'come', 'work', 'make', 'child',
        'say', 'covid', 'back', 'old', 'week', 'god', 'show', 'spiritual', 'right', 'see', 'government', 'well',
        'pandemic'
    ],
    'science': [
        'market', 'research', 'company', 'report', 'space', 'device', 'system', 'science', 'global', 'product', 'data',
        'technology', 'year', 'study', 'industry', 'work', 'make', 'medical', 'service', 'program', 'design',
        'business', 'restaurant', 'patient', 'life'
    ],
    'social': [
        'health', 'case', 'life', 'social', 'abortion', 'government', 'community', 'work', 'need', 'service', 'baby',
        'woman', 'covid', 'program', 'support', 'child', 'week', 'year', 'right', 'death', 'make', 'number', 'group',
        'month', 'family'
    ],
    'sport': [
        'team', 'game', 'season', 'match', 'run', 'player', 'back', 'second', 'world', 'final', 'win', 'coach', 'play',
        'league', 'championship', 'four', 'sport', 'tournament', 'best', 'right', 'open', 'round', 'event', 'fan',
        'year'
    ],
    'unrest': [
        'war', 'police', 'shooting', 'government', 'violence', 'force', 'attack', 'protest', 'say', 'gun', 'month',
        'official', 'group', 'military', 'weapon', 'year', 'place', 'president', 'part', 'right', 'come', 'old',
        'security', 'officer', 'world'
    ],
    'weather': [
        'climate', 'change', 'heat', 'weather', 'temperature', 'area', 'water', 'high', 'risk', 'carbon', 'emission',
        'degree', 'global', 'national', 'storm', 'service', 'warning', 'part', 'level', 'record', 'report', 'rain',
        'world', 'according', 'plan'
    ]
}


def check_word_match(row):
    category = row['category']
    words = set(row['text'].lower().split())  # Convert text to lowercase and split into words

    if category in checkWords:
        match_count = sum(1 for word in checkWords[category] if word in words)  # Count matches
        return match_count >= 3

    return False  # If category is not in the mapping, return False

# Apply function to create new column
df['matches'] = df.apply(check_word_match, axis=1)

def remove_false_entries(df):
  return df[df['matches'] == True]

df = remove_false_entries(df)

#This removes the other category completely
#df = df[df['category'] != 'other']

In [29]:
2#THIS CODE BLOCK COUNTS WORDS, USED TO FIND COMMON WORDS IN CATEGORIES
from collections import Counter
import pandas as pd

def count_common_words(df):
    category_word_counts = {}

    for category in df["category"].unique():
        # Get all text for this category
        category_text = " ".join(df[df["category"] == category]["clean_text"])

        # Tokenize and count word frequencies
        words = category_text.split()
        word_counts = Counter(words)

        # Store the word frequency dictionary
        category_word_counts[category] = dict(word_counts)

    return category_word_counts

# Get word counts per category
word_counts_by_category = count_common_words(df)

# Display the top 10 most common words for each category
for category, word_counts in word_counts_by_category.items():
    print(f"Category: {category}")
    print(dict(Counter(word_counts).most_common(10)))  # Show top 10 words
    print("-" * 40)

print(df)

Category: arts
{'show': 268, 'music': 250, 'event': 238, 'series': 205, 'park': 188, 'group': 163, 'take': 158, 'million': 155, 'community': 154, 'art': 151}
----------------------------------------
Category: crime
{'police': 552, 'court': 380, 'case': 255, 'law': 237, 'officer': 222, 'right': 221, 'year': 179, 'charge': 171, 'prison': 170, 'member': 169}
----------------------------------------
Category: disaster
{'fire': 587, 'water': 368, 'area': 224, 'police': 213, 'air': 202, 'rescue': 162, 'service': 157, 'community': 153, 'near': 136, 'official': 135}
----------------------------------------
Category: economy
{'company': 348, 'market': 245, 'price': 230, 'per': 220, 'million': 199, 'tax': 194, 'share': 162, 'store': 162, 'billion': 150, 'business': 147}
----------------------------------------
Category: education
{'student': 1316, 'school': 1292, 'university': 772, 'education': 521, 'high': 322, 'college': 302, 'teacher': 300, 'program': 298, 'community': 271, 'learning': 267}
-

In [30]:
nltk.download("averaged_perceptron_tagger_eng")

def extract_nouns_verbs(text):
  tokens = word_tokenize(text)
  pos_tags = nltk.pos_tag(tokens)
  return " ".join([word for word, tag in pos_tags if tag in ["NN", "NNS", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "JJ", "RB", "IN"]])
  #return " ".join([word for word, tag in pos_tags if tag in ["NN", "NNS", "NNP", "NNPS"]])


df["features"] = df["clean_text"].apply(extract_nouns_verbs)
print(df)
df.to_csv("data_features.csv", index=False)

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


     category                                               text  \
0        arts  rob delaney vir das galen hopper samson kayo g...   
1        arts  andris nelsons conducts a joint concert of the...   
2        arts  warner music group has brought on sherry tan t...   
3        arts  adele will explore what she s been going throu...   
4        arts  you are using an older browser version. please...   
...       ...                                                ...   
5287  weather  orangeburg . this evening in orangeburg cloudy...   
5288  weather  national basketball association north american...   
5289  weather  las vegas ap the southwest continued to bake s...   
5290  weather  here is today s weather outlook for jun. 25 20...   
5291  weather  singapore about 10000 years ago sea levels in ...   

                                             clean_text  matches  \
0     rob hopper samson kayo guz khan nick ross lee ...     True   
1     nelson conduct joint concert boston symph

In [31]:
#assign columns to variables
X = df["features"]
Y = df["category"]
print(Y.unique())

['arts' 'crime' 'disaster' 'economy' 'education' 'environmental' 'health'
 'humanInterest' 'labour' 'lifestyle' 'other' 'politics' 'religion'
 'science' 'social' 'sport' 'unrest' 'weather']


In [32]:
tfidf_vectorizer = TfidfVectorizer(max_features=200000, ngram_range=(1,2), min_df=3, max_df=0.8, stop_words='english')
X_tfidf = tfidf_vectorizer.fit_transform(X)

In [20]:
from ast import mod
#split data
x_train, x_test, y_train, y_test = train_test_split(X_tfidf, Y, test_size=0.2, random_state=42)

# Logistic Regression Model
log_model = LogisticRegression(C=1.0, max_iter=5000, class_weight="balanced")
log_model.fit(x_train, y_train)

# Random Forest Model (Fixed)
rf_model = RandomForestClassifier(n_estimators=200, class_weight="balanced", random_state=42)
rf_model.fit(x_train, y_train)

# Support Vector Machine Model
svm_model = SVC(kernel="linear", class_weight="balanced", random_state=42)
svm_model.fit(x_train, y_train)

SVC(class_weight='balanced', kernel='linear', random_state=42)

In [21]:
#test the model
y_pred_log = log_model.predict(x_test)
y_pred_rf = rf_model.predict(x_test)
y_pred_svm = svm_model.predict(x_test)

In [22]:
#print test results
print(metrics.classification_report(y_test, y_pred_log))
accuracy_log = log_model.score(x_test, y_test)

print(metrics.classification_report(y_test, y_pred_rf))
accuracy_rf = rf_model.score(x_test, y_test)

print(metrics.classification_report(y_test, y_pred_svm))
accuracy_rf = svm_model.score(x_test, y_test)

               precision    recall  f1-score   support

         arts       0.57      0.53      0.55        51
        crime       0.70      0.78      0.74        49
     disaster       0.50      0.54      0.52        46
      economy       0.68      0.58      0.63        36
    education       0.79      0.92      0.85        52
environmental       0.83      0.78      0.80        49
       health       0.61      0.58      0.59        52
humanInterest       0.70      0.65      0.68        40
       labour       0.71      0.74      0.73        47
    lifestyle       0.51      0.50      0.51        40
        other       0.00      0.00      0.00        32
     politics       0.41      0.40      0.40        45
     religion       0.52      0.37      0.43        43
      science       0.48      0.67      0.56        36
       social       0.51      0.38      0.43        48
        sport       0.52      0.86      0.65        42
       unrest       0.71      0.77      0.74        31
      wea

In [24]:
import builtins
def predict_category(text):
    pred = log_model.predict([text])[0]
    return pred

textin = builtins.input("Enter text: ")

print(predict_category(textin))

Enter text: many people died


ValueError: Expected 2D array, got 1D array instead:
array=['many people died'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.